### POP test dodecahedron

In [1]:
include("LiPoSID.jl")
using QuantumOptics
basis = NLevelBasis(2)
using DynamicPolynomials
using LinearAlgebra
using Dates
using HDF5

┌ Info: Precompiling TSSOS [81648402-ffe6-11e9-2394-0de17a9afdad]
└ @ Base loading.jl:1423
┌ Warning: Package TSSOS does not have Printf in its dependencies:
│ - If you have TSSOS checked out for development and have
│   added Printf as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with TSSOS
└ Loading Printf into TSSOS from project dependency, future warnings for TSSOS are suppressed.


In [2]:
function get_operator(file, gamma_group, operator_name)
    h5open(file,"r") do fid # read-only
        A = read(fid[gamma_group][operator_name])
        return convert(Matrix{ComplexF64}, A)
    end
end

get_operator (generic function with 1 method)

In [3]:
parentdir = pwd()
parentdir = "C:\\Users\\Zakhar\\Documents\\GitHub\\POP23\\"
data_dir = parentdir*"\\DATA\\"
println(data_dir)
models_dir = parentdir*"\\MODELS\\"
tests_dir = parentdir*"\\TESTS\\"

C:\Users\Zakhar\Documents\GitHub\POP23\\DATA\


"C:\\Users\\Zakhar\\Documents\\GitHub\\POP23\\\\TESTS\\"

Choose last models file

In [4]:
cd(models_dir)
models_file = ""
for (root, dirs, files) in walkdir(".") 
    models_file = files[end]
end
println(models_file)

POP_Tr0_train-REG-B_2023-Jul-07_at_10-52.h5


In [5]:
test_files = ["State_D"*string(n) for n=1:20];

$A' = A + a I$

$ a = -\operatorname{tr}(A)/2$

$ H' = H + \frac{1}{2i} (a^*A - a A^\dagger)$

### Test models loop

In [6]:
date_and_time_string =  string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))
tests_data_file_name = "POP_Tr0_test-B_" * date_and_time_string * ".h5"

println("Test to be saved in ", tests_data_file_name)

γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

for γᵢ in γ
    
    println(" γ = ", γᵢ)
    
    h5open(tests_dir*tests_data_file_name,"cw") do fid
                γ_group = create_group(fid, "gamma_"*γᵢ)
    end 
    
    H = DenseOperator(basis,get_operator(models_dir*models_file, "gamma_"*γᵢ, "H"))
    A = DenseOperator(basis,get_operator(models_dir*models_file, "gamma_"*γᵢ, "A"))
            
    for df in test_files # loop over initial states

                print(df*" ")
        
                start_time = time()

                ρ, t = LiPoSID.get_rho_series(data_dir*df*"_2CUT_data.h5", γᵢ)
                ρ = convert(Vector{Matrix{ComplexF64}}, ρ)
               
                ρᵗᵉˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρ]
        
                t = convert(Vector{Float64}, t)
            
                tᴸᴹᴱ, ρᴸᴹᴱ  = timeevolution.master(t, ρᵗᵉˢᵗ[1], H, [A])
            
                F = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρᵗᵉˢᵗ, ρᴸᴹᴱ)]
                
                runtime = time() - start_time
                
                h5open(tests_dir*tests_data_file_name,"cw") do fid
                    γ_group = open_group(fid, "gamma_"*γᵢ) # open coupling group

                    init_state_group = create_group(γ_group, df) # create initial state group
                    init_state_group["fidelity"] = convert.(Float64, F)
                    init_state_group["runtime"] = runtime
                end
    
    end
                
    println()
    
end 

┌ Warning: Backwards compatability support of the new return codes to Symbols will be deprecated with the Julia v1.9 release. Please see https://docs.sciml.ai/SciMLBase/stable/interfaces/Solutions/#retcodes for more information
└ @ SciMLBase C:\Users\Zakhar\.julia\packages\SciMLBase\QqtZA\src\retcodes.jl:355
┌ Warning: Backwards compatability support of the new return codes to Symbols will be deprecated with the Julia v1.9 release. Please see https://docs.sciml.ai/SciMLBase/stable/interfaces/Solutions/#retcodes for more information
└ @ SciMLBase C:\Users\Zakhar\.julia\packages\SciMLBase\QqtZA\src\retcodes.jl:355
┌ Warning: Backwards compatability support of the new return codes to Symbols will be deprecated with the Julia v1.9 release. Please see https://docs.sciml.ai/SciMLBase/stable/interfaces/Solutions/#retcodes for more information
└ @ SciMLBase C:\Users\Zakhar\.julia\packages\SciMLBase\QqtZA\src\retcodes.jl:355
┌ Warning: Backwards compatability support of the new return codes to 

┌ Warning: Backwards compatability support of the new return codes to Symbols will be deprecated with the Julia v1.9 release. Please see https://docs.sciml.ai/SciMLBase/stable/interfaces/Solutions/#retcodes for more information
└ @ SciMLBase C:\Users\Zakhar\.julia\packages\SciMLBase\QqtZA\src\retcodes.jl:355
┌ Warning: Backwards compatability support of the new return codes to Symbols will be deprecated with the Julia v1.9 release. Please see https://docs.sciml.ai/SciMLBase/stable/interfaces/Solutions/#retcodes for more information
└ @ SciMLBase C:\Users\Zakhar\.julia\packages\SciMLBase\QqtZA\src\retcodes.jl:355
┌ Warning: Backwards compatability support of the new return codes to Symbols will be deprecated with the Julia v1.9 release. Please see https://docs.sciml.ai/SciMLBase/stable/interfaces/Solutions/#retcodes for more information
└ @ SciMLBase C:\Users\Zakhar\.julia\packages\SciMLBase\QqtZA\src\retcodes.jl:355
┌ Warning: Backwards compatability support of the new return codes to 

┌ Warning: Backwards compatability support of the new return codes to Symbols will be deprecated with the Julia v1.9 release. Please see https://docs.sciml.ai/SciMLBase/stable/interfaces/Solutions/#retcodes for more information
└ @ SciMLBase C:\Users\Zakhar\.julia\packages\SciMLBase\QqtZA\src\retcodes.jl:355
┌ Warning: Backwards compatability support of the new return codes to Symbols will be deprecated with the Julia v1.9 release. Please see https://docs.sciml.ai/SciMLBase/stable/interfaces/Solutions/#retcodes for more information
└ @ SciMLBase C:\Users\Zakhar\.julia\packages\SciMLBase\QqtZA\src\retcodes.jl:355
┌ Warning: Backwards compatability support of the new return codes to Symbols will be deprecated with the Julia v1.9 release. Please see https://docs.sciml.ai/SciMLBase/stable/interfaces/Solutions/#retcodes for more information
└ @ SciMLBase C:\Users\Zakhar\.julia\packages\SciMLBase\QqtZA\src\retcodes.jl:355
┌ Warning: Backwards compatability support of the new return codes to 

┌ Warning: Backwards compatability support of the new return codes to Symbols will be deprecated with the Julia v1.9 release. Please see https://docs.sciml.ai/SciMLBase/stable/interfaces/Solutions/#retcodes for more information
└ @ SciMLBase C:\Users\Zakhar\.julia\packages\SciMLBase\QqtZA\src\retcodes.jl:355
┌ Warning: Backwards compatability support of the new return codes to Symbols will be deprecated with the Julia v1.9 release. Please see https://docs.sciml.ai/SciMLBase/stable/interfaces/Solutions/#retcodes for more information
└ @ SciMLBase C:\Users\Zakhar\.julia\packages\SciMLBase\QqtZA\src\retcodes.jl:355
┌ Warning: Backwards compatability support of the new return codes to Symbols will be deprecated with the Julia v1.9 release. Please see https://docs.sciml.ai/SciMLBase/stable/interfaces/Solutions/#retcodes for more information
└ @ SciMLBase C:\Users\Zakhar\.julia\packages\SciMLBase\QqtZA\src\retcodes.jl:355
┌ Warning: Backwards compatability support of the new return codes to 

Test to be saved in POP_Tr0_test-B_2023-Jul-07_at_11-21.h5
 γ = 0.079477
State_D1 State_D2 State_D3 State_D4 State_D5 State_D6 State_D7 State_D8 State_D9 State_D10 State_D11 State_D12 State_D13 State_D14 State_D15 State_D16 State_D17 State_D18 State_D19 State_D20 
 γ = 0.25133
State_D1 State_D2 State_D3 State_D4 State_D5 State_D6 State_D7 State_D8 State_D9 State_D10 State_D11 State_D12 State_D13 State_D14 State_D15 State_D16 State_D17 State_D18 State_D19 State_D20 
 γ = 0.79477
State_D1 State_D2 State_D3 State_D4 State_D5 State_D6 State_D7 State_D8 State_D9 State_D10 State_D11 State_D12 State_D13 State_D14 State_D15 State_D16 State_D17 State_D18 State_D19 State_D20 
 γ = 2.5133
State_D1 State_D2 State_D3 State_D4 State_D5 State_D6 State_D7 State_D8 State_D9 State_D10 State_D11 State_D12 State_D13 State_D14 State_D15 State_D16 State_D17 State_D18 State_D19 State_D20 
 γ = 7.9477
State_D1 State_D2 State_D3 State_D4 State_D5 State_D6 State_D7 State_D8 State_D9 State_D10 State_D11 State_D1

┌ Warning: Backwards compatability support of the new return codes to Symbols will be deprecated with the Julia v1.9 release. Please see https://docs.sciml.ai/SciMLBase/stable/interfaces/Solutions/#retcodes for more information
└ @ SciMLBase C:\Users\Zakhar\.julia\packages\SciMLBase\QqtZA\src\retcodes.jl:355
┌ Warning: Backwards compatability support of the new return codes to Symbols will be deprecated with the Julia v1.9 release. Please see https://docs.sciml.ai/SciMLBase/stable/interfaces/Solutions/#retcodes for more information
└ @ SciMLBase C:\Users\Zakhar\.julia\packages\SciMLBase\QqtZA\src\retcodes.jl:355
┌ Warning: Backwards compatability support of the new return codes to Symbols will be deprecated with the Julia v1.9 release. Please see https://docs.sciml.ai/SciMLBase/stable/interfaces/Solutions/#retcodes for more information
└ @ SciMLBase C:\Users\Zakhar\.julia\packages\SciMLBase\QqtZA\src\retcodes.jl:355
┌ Warning: Backwards compatability support of the new return codes to 